<a href="https://colab.research.google.com/github/Knight7561/hf-smol-course/blob/learn/1_instruction_tuning/student_examples/adamjweintraut-dataset/eli5_precomputed_top_slice/sft_finetuning_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Supervised Fine-Tuning with SFTTrainer

This notebook demonstrates how to fine-tune the `HuggingFaceTB/SmolLM2-135M` model using the `SFTTrainer` from the `trl` library. The notebook cells run and will finetune the model. You can select your difficulty by trying out different datasets.

<div style='background-color: lightblue; padding: 10px; border-radius: 5px; margin-bottom: 20px; color:black'>
    <h2 style='margin: 0;color:blue'>Exercise: Fine-Tuning SmolLM2 with SFTTrainer</h2>
    <p>Take a dataset from the Hugging Face hub and finetune a model on it. </p>
    <p><b>Difficulty Levels</b></p>
    <p>🐢 Use the `HuggingFaceTB/smoltalk` dataset</p>
    <p>🐕 Try out the `bigcode/the-stack-smol` dataset and finetune a code generation model on a specific subset `data/python`.</p>
    <p>🦁 Select a dataset that relates to a real world use case your interested in</p>
</div>

In [ ]:
# Install the requirements in Google Colab
!pip install transformers datasets trl huggingface_hub

# Authenticate to Hugging Face

from huggingface_hub import login
login()

# for convenience you can create an environment variable containing your hub token as HF_TOKEN

In [ ]:
import os
os.environ['WANDB_DISABLED'] = 'true'

In [ ]:
# Import necessary libraries
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset
from trl import SFTConfig, SFTTrainer, setup_chat_format
import torch

device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps" if torch.backends.mps.is_available() else "cpu"
)

# Load the model and tokenizer
model_name = "HuggingFaceTB/SmolLM2-135M"
model = AutoModelForCausalLM.from_pretrained(
    pretrained_model_name_or_path=model_name
).to(device)
tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name_or_path=model_name)

# Set up the chat format
model, tokenizer = setup_chat_format(model=model, tokenizer=tokenizer)

# Set our name for the finetune to be saved &/ uploaded to
finetune_name = "SmolLM2-eli5_precomputed_top_slice"
finetune_tags = ["smol-course-learn", "module_1","eli5_precomputed_top_slice"]

config.json:   0%|          | 0.00/704 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/269M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/3.66k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/801k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/466k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.10M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/831 [00:00<?, ?B/s]

# Generate with the base model

Here we will try out the base model which does not have a chat template.

In [ ]:
# Let's test the base model before training
prompt = "Write a haiku about programming"

# Format with template
messages = [{"role": "user", "content": prompt}]
formatted_prompt = tokenizer.apply_chat_template(messages, tokenize=False)

# Generate response
inputs = tokenizer(formatted_prompt, return_tensors="pt").to(device)
outputs = model.generate(**inputs, max_new_tokens=100)
print("Before training:")
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Before training:
user
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a


## Dataset Preparation

We will load a sample dataset and format it for training. The dataset should be structured with input-output pairs, where each input is a prompt and the output is the expected response from the model.

**TRL will format input messages based on the model's chat templates.** They need to be represented as a list of dictionaries with the keys: `role` and `content`,.

In [ ]:
# Load a sample dataset
from datasets import load_dataset

# TODO: define your dataset and config using the path and name parameters
ds = load_dataset(path="adamjweintraut/eli5_precomputed_top_slice")

README.md:   0%|          | 0.00/881 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/172M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/21.4M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/21.5M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/10000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1250 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1250 [00:00<?, ? examples/s]

In [ ]:
ds

DatasetDict({
    train: Dataset({
        features: ['index', 'q_id', 'question', 'best_answer', 'all_answers', 'num_answers', 'top_answers', 'num_top_answers', 'context', 'orig', 'target'],
        num_rows: 10000
    })
    test: Dataset({
        features: ['index', 'q_id', 'question', 'best_answer', 'all_answers', 'num_answers', 'top_answers', 'num_top_answers', 'context', 'orig', 'target'],
        num_rows: 1250
    })
    validation: Dataset({
        features: ['index', 'q_id', 'question', 'best_answer', 'all_answers', 'num_answers', 'top_answers', 'num_top_answers', 'context', 'orig', 'target'],
        num_rows: 1250
    })
})

In [ ]:
# TODO: 🦁 If your dataset is not in a format that TRL can convert to the chat template, you will need to process it. Refer to the [module](../chat_templates.md)
def add_chat_template(row):
  messages=[
        {"role": "user", "content": row['question']},
        {"role": "assistant", "content": row['best_answer']}
    ]
  row['tokenized_message']=tokenizer.apply_chat_template(messages,tokenize=False)
  return row

ds=ds.map(add_chat_template)

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1250 [00:00<?, ? examples/s]

Map:   0%|          | 0/1250 [00:00<?, ? examples/s]

In [ ]:
# tokenizer.decode(ds['train'][3]["tokenized_message"])

In [ ]:
ds['train'][3]['tokenized_message']

'<|im_start|>user\nhow do/did pirate radio stations get shut down?<|im_end|>\n<|im_start|>assistant\nProbably using a dish antenna, like satellite TV uses, and triangulation. The antenna picks up signal strongest from where it is pointing at, so they take three readings from three different places and trace the lines back to the source. Assuming that the signal is all they have to go on, this might take several tries, as the method is inaccurate from a long way off. But if they have some other clues that they could use to guess the general location of the broadcaster, then the amount of tries drops to as few as one.<|im_end|>\n'

## Configuring the SFTTrainer

The `SFTTrainer` is configured with various parameters that control the training process. These include the number of training steps, batch size, learning rate, and evaluation strategy. Adjust these parameters based on your specific requirements and computational resources.

In [ ]:
# Configure the SFTTrainer
sft_config = SFTConfig(
    output_dir="./sft_output",
    max_steps=1000,  # Adjust based on dataset size and desired training duration
    per_device_train_batch_size=4,  # Set according to your GPU memory capacity
    learning_rate=5e-5,  # Common starting point for fine-tuning
    logging_steps=10,  # Frequency of logging training metrics
    save_steps=100,  # Frequency of saving model checkpoints
    eval_strategy="steps",  # Evaluate the model at regular intervals
    eval_steps=50,  # Frequency of evaluation
    use_mps_device=(
        True if device == "mps" else False
    ),  # Use MPS for mixed precision training
    hub_model_id=finetune_name,  # Set a unique name for your model
    report_to=None,
    max_seq_length=1024,
    dataset_text_field="tokenized_message"
)

# Initialize the SFTTrainer
trainer = SFTTrainer(
    model=model,
    args=sft_config,
    train_dataset=ds["train"],
    tokenizer=tokenizer,
    eval_dataset=ds["test"]
)

# TODO: 🦁 🐕 align the SFTTrainer params with your chosen dataset. For example, if you are using the `bigcode/the-stack-smol` dataset, you will need to choose the `content` column`

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
max_steps is given, it will override any value given in num_train_epochs


## Training the Model

With the trainer configured, we can now proceed to train the model. The training process will involve iterating over the dataset, computing the loss, and updating the model's parameters to minimize this loss.

In [ ]:
# Train the model
trainer.train()

# Save the model
trainer.save_model(f"./{finetune_name}")

Step,Training Loss,Validation Loss
50,2.828000,2.882840
100,2.837100,2.870539
150,2.863800,2.866266
200,2.878600,2.862974
250,2.765300,2.861464
300,2.995700,2.859624
350,2.839100,2.858479
400,2.871400,2.856515
450,2.782200,2.854177
500,2.856500,2.852748


KeyboardInterrupt: 

The above training was stopped because there was no apperciable improvement observed in the epochs.

In [ ]:
trainer

In [ ]:
# Set our name for the finetune to be saved &/ uploaded to
trainer.push_to_hub(tags=finetune_tags)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


model.safetensors:   0%|          | 0.00/538M [00:00<?, ?B/s]

events.out.tfevents.1734107976.995ff5f4afa7.3375.0:   0%|          | 0.00/5.56k [00:00<?, ?B/s]

events.out.tfevents.1734108061.995ff5f4afa7.3375.1:   0%|          | 0.00/18.8k [00:00<?, ?B/s]

Upload 4 LFS files:   0%|          | 0/4 [00:00<?, ?it/s]

training_args.bin:   0%|          | 0.00/5.56k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/knight7561/SmolLM2-eli5_precomputed_top_slice/commit/5ce5b897a18a2e0b1610e30b32eedde5356dbee6', commit_message='End of training', commit_description='', oid='5ce5b897a18a2e0b1610e30b32eedde5356dbee6', pr_url=None, repo_url=RepoUrl('https://huggingface.co/knight7561/SmolLM2-eli5_precomputed_top_slice', endpoint='https://huggingface.co', repo_type='model', repo_id='knight7561/SmolLM2-eli5_precomputed_top_slice'), pr_revision=None, pr_num=None)

<div style='background-color: lightblue; padding: 10px; border-radius: 5px; margin-bottom: 20px; color:black'>
    <h2 style='margin: 0;color:blue'>Bonus Exercise: Generate with fine-tuned model</h2>
    <p>🐕 Use the fine-tuned to model generate a response, just like with the base example..</p>
</div>

In [ ]:
# Test the fine-tuned model on the same prompt

def test_finetuned_model(prompt):
  # Let's test the base model before training

  # Format with template
  messages = [{"role": "user", "content": prompt}]
  formatted_prompt = tokenizer.apply_chat_template(messages, tokenize=False)

  # Generate response
  inputs = tokenizer(formatted_prompt, return_tensors="pt").to(device)

  # TODO: use the fine-tuned to model generate a response, just like with the base example.
  output=model.generate(**inputs,max_new_tokens=100)
  print("After training \n")
  print(tokenizer.decode(output[0], skip_special_tokens=True))

In [ ]:
test_finetuned_model("why do old cartoons look so jittery?")

After training 

user
why do old cartoons look so jittery?
assistant
It's because the cartoon is old.  It's not a new cartoon.  It's a cartoon that's been around for a long time.  It's not a new cartoon.  It's a cartoon that's been around for a long time.  It's a cartoon that's been around for a long time.  It's a cartoon that's been around for a long time.  It's a cartoon that's been around for a long time.  It


In [ ]:
test_finetuned_model("Write a function to convert degrees to radians.")

After training 

user
Write a function to convert degrees to radians.
assistant
The radian is defined as the angle subtended by an arc of a circle of radius r.  The radian is defined as the angle subtended by an arc of a circle of radius r/2.  The radian is defined as the angle subtended by an arc of a circle of radius r/2/2.  The radian is defined as the angle subtended by an arc of a circle of radius r/2/2.  The rad


In [ ]:
prompt="""why do some of us fell the urge to smile at something we know is terrible, like seeing another in pain or hearing someone has been killed, even though we know we shouldn't?"""
best_answer="Everybody responds to things in their own unique way depending on many factors throughout their life. My guess is that the behaviour of smiling when hearing of distressing news is a learned psychological coping mechanism. It could be a way to avoid inner feelings of fear and distress, especially if a person has been taught to feel shame regarding appropriate outward expression of emotional distress in their family of origin. I know a guy who laughs when he hears distressing news, it's a part of how he releases the energy of the emotions he is uncomfortable with because he was taught that crying and yelling is inappropriate. I don't know if that lands for you regarding your experience in your life, but that is my theory."
test_finetuned_model(prompt)
print("\n\n\nBest Answer:(from test dataset)\n",best_answer)

After training 

user
why do some of us fell the urge to smile at something we know is terrible, like seeing another in pain or hearing someone has been killed, even though we know we shouldn't?
assistant
I think it's because we're not used to it.  We're used to seeing people who are in pain, and we're used to hearing people who are in pain.  We're used to seeing people who are in pain, and we're used to hearing people who are in pain.  We're used to seeing people who are in pain, and we're used to hearing people who are in pain.  We're used to seeing people who are in pain,



Best Answer:(from test dataset) Everybody responds to things in their own unique way depending on many factors throughout their life. My guess is that the behaviour of smiling when hearing of distressing news is a learned psychological coping mechanism. It could be a way to avoid inner feelings of fear and distress, especially if a person has been taught to feel shame regarding appropriate outward expression of 

## 💐 You're done!

This notebook provided a step-by-step guide to fine-tuning the `HuggingFaceTB/SmolLM2-135M` model using the `SFTTrainer`. By following these steps, you can adapt the model to perform specific tasks more effectively. If you want to carry on working on this course, here are steps you could try out:

- Try this notebook on a harder difficulty
- Review a colleagues PR
- Improve the course material via an Issue or PR.